1、确认分析目标，可以稍微延伸；
    a.用户画像（4W1H描述使用场景）
    b.好评/差评（可与其他变量如星级、时间、变体、点赞数等交叉分析）
    c.购买动机（why，其他同上）
    d.技巧：CoT、few-shot learning、实体识别、情感分析（摘要、抽象）等
2、基于数据搭建流程；
    a.streamlit + 云*
    b.pyqt*
    c.BI tool
3、快速验证流程有效性

In [1]:
# 导入大模型、情感分析模型、测试数据等
from openai import OpenAI
import pandas as pd

# client_ds = OpenAI(api_key="sk-c54b0f152ba1473cbfb3ef2b0b65878b", base_url="https://api.deepseek.com/v1").chat.completions
client_ds = OpenAI(api_key="sk-c54b0f152ba1473cbfb3ef2b0b65878b", base_url="https://api.deepseek.com/beta").chat.completions # 8k输出长度
# client_kimi = OpenAI(api_key="sk-cMpxKyHfnkW4Iwr9BWWbr0M2Rg6J1tiez1sQSTSbTILaIWUF", base_url="https://api.moonshot.cn/v1").chat.completions
data1 = pd.read_excel('./B07TXYC6WD-US-Reviews-240910-73658.xlsx')
data1.columns = ['type','vp','vine','title','review','translation','rating','thumbup','picture','video','url','timing']

In [106]:
# 用户画像分析
import json
from tqdm import tqdm
from langdetect import detect
from translate import Translator
translator = Translator(from_lang='autodetect', to_lang='en')

## 数据预处理
# 数据量是否充足
# 部分非英文需要翻译
# 无效数据

def preprocessing(df):
    df_new = df.copy()
    print("原数据长度：", len(df_new))

    # 去除空字段
    not_na_idx = df_new["review"].notna()
    df_new = df_new.loc[not_na_idx]
    df_new.reset_index(drop=True, inplace=True)
    print("无空字段数据长度：", len(df_new))

    # 翻译
    title_list = df_new['title'].to_list()
    review_list = df_new['review'].to_list()
    title_list_trans = []
    review_list_trans = []
    title_trans_count = 0
    review_trans_count = 0
    title_error_idx = []
    review_error_idx = []

    # for idx, title_review in tqdm(enumerate(zip(title_list, review_list))):
    #     title = title_review[0]
    #     review = title_review[1]

    for idx, row in tqdm(df_new.iterrows()): # 与多线程测试速度
        title = row['title']
        review = row['review']

        try:
            title_lang = detect(title)
        except Exception as e:
            print(f'索引：{idx}；类型：title；内容：{title}；错误：{e}')
            title_error_idx.append(idx)
            title_lang = None

        if title_lang == 'en' or title_lang is None:
            title_list_trans.append(title)
        else:
            title_trans = translator.translate(title)
            if title_trans == "PLEASE SELECT TWO DISTINCT LANGUAGES":
                title_list_trans.append(title)
            else:
                title_list_trans.append(title_trans)
                title_trans_count += 1        

        try:
            review_lang = detect(review)
        except Exception as e:
            print(f'索引：{idx}出错；类型：review；内容：{review}；错误：{e}')
            review_error_idx.append(idx)
            review_lang = None

        if review_lang == 'en' or review_lang is None:
            review_list_trans.append(review)
        else:
            review_trans = translator.translate(review)
            if review_trans == "PLEASE SELECT TWO DISTINCT LANGUAGES":
                review_list_trans.append(review)
            else:
                review_list_trans.append(review_trans)
                review_trans_count += 1    

    print(f"标题处理的数量：{title_trans_count}")
    print(f"评论处理的数量：{review_trans_count}")

    df_new['title'] = title_list_trans
    df_new['review'] = review_list_trans

    df_new.drop(title_error_idx, inplace = True, axis = 0)
    df_new.drop(review_error_idx, inplace = True, axis = 0)
    df_new.reset_index(drop=True, inplace = True)
    print(f"最终处理数据长度：{len(df_new)}")

    return df_new

data1_processed = preprocessing(data1.head(idx)) # 测试速度
# data1_processed.to_excel('./data1_processed.xlsx', index=False)

原数据长度： 50
无空字段数据长度： 50


50it [00:40,  1.22it/s]

标题处理的数量：5
评论处理的数量：4
最终处理数据长度：50


In [51]:
## 画像形成/评论分析：大模型实体抽取4W1H和购买动机，计算比例
# 什么时间购买和使用：生日、日常游玩还是睡前游玩等
# 什么场景：在哪里使用？家中还是车上等；
# 用户关注点以及如何使用：产品特性、教学工具等
# 用户：谁用、谁买
# how：如何使用、如何发现
# 动机：礼物、教育等 ？
# 评价：是否好评或差点，原因是什么？

def get_entity(title, review):
    # entity extraction
    role_function = """
            You are a helpful expert for entity extraction, and are capable of performing the task below.
            
            You are required to extract key entities and summarize the text content from the Amazon product review. 

            Key entities include:
            1. Product purchase time (e.g., around birthday, before/after holidays, etc.)
            2. Usage scenario (e.g., birthday party, gift, workplace, etc.)
            3. Target user (e.g., children, elderly, etc.)
            4. Purchase motive (e.g., for education, as a gift)
            5. Information channel for discovering the product (e.g., recommended by others, redirected from other websites, etc.)

            If there is no corresponding entity in the text, return an empty value.

            The summary of the text content should include:
            1. User's evaluation (positive, negative or neutral)
            2. Reason for the positive or negative review
            3. Product features that the user focuses on (e.g., color, size, etc.)

            Return the results in JSON format and lower case.

            Example text:
            "I bought this toy before Christmas as a gift for my nephew. He had a lot of fun with it at his birthday party, especially liking its color and size. I discovered this product through a friend's recommendation."

            JSON format return result example:
            {
            "entities": {
                "purchase time": "before Christmas",
                "usage scenario": "birthday party",
                "target user": "nephew",
                "purchase motive": "as a gift",
                "information channel": "friend's recommendation"
            },
            "summary": {
                "evaluation": "positive",
                "reason": "nephew had a lot of fun at the birthday party",
                "focus on product features": "color and size"
            }
            }
            """
    messages = [{"role": "system", "content": role_function},
                {"role": "user", "content": f"Please process the following review: title - {title}; review - {review}"}]
    response = client_ds.create(
                                model = "deepseek-chat",
                                messages = messages,
                                temperature = 0.1,
                                response_format={'type':"json_object"})
    entity_set = json.loads(response.choices[0].message.content)
    return entity_set


# idx = np.random.choice(data1_processed.index)
# title = data1_processed['title'][idx]
# review = data1_processed['review'][idx]
# print(title)
# print(review)
# review_analysis = get_entity(title, review)
# review_analysis

def get_entity_for_all(data):
    title_list = data['title'].to_list()
    review_list = data['review'].to_list()

    review_analysis = []
    error_index = []    
    for idx, title_review in tqdm(enumerate(zip(title_list, review_list))):
        title = title_review[0]
        review = title_review[1]

        try:
            result = get_entity(title, review)
            review_analysis.append(result)
        except Exception as e:
            print(f'索引：{idx}出错；内容：{title} {review}；错误：{e}')
            review_analysis.append({})
            error_index.append(idx)
        
    return review_analysis, error_index


review_analysis, error_index = get_entity_for_all(data1_processed)

#### json验证
#### 并发询问

# import json
# with open('product1_review_test_entity.json', 'w', encoding='utf-8') as json_file:
#     for i in review_analysis:
#         json.dump(i, json_file, ensure_ascii=False, indent=4)

# data1_processed['review_analysis'] = review_analysis

0it [00:00, ?it/s]

699it [1:12:27,  6.22s/it]


In [2]:
import json

with open("./product1_review_test_entity.json", 'r', encoding='utf-8') as file:
    review_analysis = json.load(file)

In [4]:
# statistical analysis
def analyze_entity(entity_set):
    # 实体分析
    time = []
    scenario = []
    user = []
    motive = []
    channel = []
    evaluation = []
    reason = []
    feature = []
    for subset in entity_set: #### 修改为列表
        entities = subset['entities']
        summary = subset['summary']
        time.append(entities['purchase time'])
        scenario.append(entities['usage scenario'])
        user.append(entities['target user'])
        motive.append(entities['purchase motive'])
        channel.append(entities['information channel'])
        evaluation.append(summary['evaluation'])
        reason.append(summary['reason'])
        feature.append(summary['focus on product features'])    
    
    ## 简单分析：每类实体的分布情况/基于模型总结 #### 加入评价分类、时间、退货
    entity_set_list = [time,scenario,user,motive,channel,evaluation,reason,feature]
    entity_set_filter = [[item for item in entity_type if len(item) > 0] for entity_type in entity_set_list]
    print(f"数据长度：{len(entity_set)}")
    print(f"长度：time {len(entity_set_filter[0])}、scenario {len(entity_set_filter[1])}、user {len(entity_set_filter[2])}、motive {len(entity_set_filter[3])}、"\
        f"channel {len(entity_set_filter[4])}、evaluation {len(entity_set_filter[5])}、reason {len(entity_set_filter[6])}、feature {len(entity_set_filter[7])}")
    
    return entity_set_filter, evaluation, reason
    

    #### 如何减少计算量
    ## 交叉分析：实体的交互情况
    ## 星级、变体、点赞数

entity_set_filter, evaluation, reason = analyze_entity(review_analysis)

数据长度：699
长度：time 25、scenario 93、user 427、motive 139、channel 10、evaluation 698、reason 697、feature 484


In [19]:
## 回归数据本身
# from collections import Counter
# specific_entity = entity_set_filter[1]
# print(Counter(specific_entity))
# sorted(specific_entity)

## 需要做统一
# 分类（意思相近、长相类似，而不是基于属性） -> 计算
## jaccard算法：基于token
### 需要聚类：time、scenario、user、motive、reason、feature
### 直接展示：channel、evaluation
### 选取中心词：随机 or 大模型拟定（evaluation）
## 计算：每个属性的每个聚类占比 + 文字描述即可

import string
from copy import deepcopy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_punct(text):
    # 使用string.punctuation获取所有英文标点符号
    translator = str.maketrans('', '', string.punctuation)
    # 去除标点符号
    text_without_punct = text.translate(translator)
    return text_without_punct

def remove_stopwords(ls):
    return [item for item in ls if item not in stop_words]

def jaccard_similarity(set1, set2):
    set1 = set(set1)
    set2 = set(set2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if union == 0:
        return 0
    return intersection / union

def group_entity(entity_set, threshold=0.5):
    entity_set = deepcopy(entity_set)
    if len(entity_set) == 0:
        return {}
    else:
        # 去除标点符号
        entity_set_clean = list(map(remove_punct, entity_set))
        # 分词
        entity_set_token = list(map(word_tokenize, entity_set_clean))
        # 去除停用词
        entity_set_no_stopwords = list(map(remove_stopwords, entity_set_token))
        # 形成对应关系
        # entity_set_dict = dict(zip(entity_set, entity_set_no_stopwords))

        # 分类/中心词选定
                    
    return entity_set_no_stopwords

# specific_entity_group = group_entity(specific_entity)

In [300]:
import spacy
import numpy as np
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

def cosine_similarity(vector_a, vector_b):
    # 计算点积
    dot_product = np.dot(vector_a, vector_b)
    
    # 计算向量的模
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    
    # 计算余弦相似度
    similarity = dot_product / (norm_a * norm_b)                                                                            
    
    return similarity

# 加载spaCy模型
## 不同模型（sm、md）
## 不同阈值（0.5、0.6、0.7、0.8、0.9）
## 速度尚可、效果不行 time（聚类效果可能还不如jaccard）、scenario（效果也不好）、user（不太适合统一）、motive（形式太多不适合统一）、reason（形式太多不适合统一）、feature（聚类效果不好）

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")

specific_entity = list(set(entity_set_filter[-1]))
threshold = 0.7

def aggregate_similar_phrases(phrases, threshold=0.5):
    # 批量转换
    docs = list(map(nlp, phrases))
    phase_docs = dict(zip(phrases, docs))

    # 创建一个字典来存储聚合结果
    aggregated_phrases_temp = defaultdict(dict)

    # 遍历每个短语
    for phrase, doc in phase_docs.items():
        # 初始化一个标志来表示是否找到匹配
        found_match = False

        # 遍历已有的聚合组
        for key, group in aggregated_phrases_temp.items():
            # 计算当前短语与聚合组中第一个短语的相似度
            similarity = cosine_similarity(doc.vector, group['avg_vec']) # 组平均向量作为标准

            # 如果相似度超过某个阈值（例如0.8），则认为它们是相似的
            if similarity >=  threshold:
                group[phrase] = doc.vector
                group['avg_vec'] = sum(y for x, y in group.items() if x != 'avg_vec')/(len(group) - 1) # 更新组平均向量
                found_match = True
                break

        # 如果没有找到匹配，则创建一个新的聚合组
        if not found_match:
            aggregated_phrases_temp[phrase][phrase] = doc.vector
            aggregated_phrases_temp[phrase]['avg_vec'] = doc.vector
    
    keys = list(aggregated_phrases_temp.keys())
    groups = [[__ for __ in _ if __ != 'avg_vec'] for _ in aggregated_phrases_temp.values()]
    aggregated_phrases = dict(zip(keys, groups))

    return aggregated_phrases

aggregated_phrases = aggregate_similar_phrases(specific_entity, threshold=threshold)

# 打印聚合结果
for idx, key_group in enumerate(aggregated_phrases.items()):
    print(rf"Group {idx+1}: {key_group[0]}")
    print(rf"Phrases {idx+1}: {key_group[1]}")
    print(rf"=================================")

Group 1: set of books
Phrases 1: ['set of books', 'number of books, photo representation', 'country of origin and washing instructions', 'lots of pages', 'number of books and washing instructions', 'content and instructional nature', 'softness, durability, and ease of washing', 'sturdiness, ease of cleaning', 'colour and value', 'sensory perception, selling technique', 'color, worth of money', 'vivid color, drawing of different animals, fabric tails with various textures', 'description and price', 'quality, machine washable, attachment to paci clip', 'bright colours and clear illustrations', 'quantity and size', 'colors, tails security', 'tails of each animal', 'value for the money', 'durability, noise level', 'textures, noises, and colors', 'colours and educational content about animals', 'quality, product description, wording, pages with crinkle', 'colorful, good quality', 'quality and color', 'tails sticking out from each page, crinkly pages, ties in the center of the book, quality 

In [354]:
def get_cluster(name, entitty_set):
    # entity clustering
    enetity_set_str = "、".join(entitty_set)
    role_function = '''
                    角色说明：
                        你是一个擅长数据分析的数据分析师，现在你需要完成以下任务。

                    任务说明：
                        我会给你某一产品用户购买特征的一系列描述（如，购买时间、使用场景等），请将相似的描述归类，并统计每个类别的数量。
                    
                    操作步骤：
                    1. 阅读并理解每条描述。
                    2. 根据描述内容，识别并归类相似的描述。
                    3. 对每个类别进行计数。
                    4. 输出每个类别、对应的数量以及数个例子。

                    注意事项：
                    1. 归类时主要考虑描述的相似性（比如说“1st birthday”和“for his 1st birthday”可以归为“生日”类别，“the number of books”和“quantity”可以归类为商品数量）。
                    2. 类别名称不能过于笼统，比如不能出现“书籍特征”这种类别，因为无法从“书籍特征”中得知具体的书籍特征。
                    3. 输出的例子个数为2~10个左右，注意不能重复输出。
                    4. 统计时注意不要遗漏或重复计数，总量要与输入一致。
                    5. 返回json格式。  

                    示例输入：
                            ```                                                
                            shower,
                            a few months ago,
                            for his 1st birthday,
                            during Easter2023,
                            Easter,
                            first Christmas,
                            for Christmas,
                            when baby #2 was 4 months and daughter was 2yrs,
                            baby’s first Christmas,
                            a few days ago,
                            since Christmas,
                            at Christmas,
                            birthday,
                            for a Christmas present,
                            for Christmas,
                            early Christmas,
                            1st birthday,
                            1st birthday,
                            1st birthday,
                            Christmas,
                            1st birthday,
                            Christmas,
                            for Christmas,
                            when she was 5 1/2 months old,
                            when he was about 3 months
                            ```

                    示例输出：
                            ```
                            {
                            "生日": {"数量":7,"例子":["1st birthday","birthday",...]}, 
                            "圣诞节": {"数量":10,"例子":["for Christmas","Christmas",...]},
                            "宝宝成长": {"数量":2,"例子":["when he was about 3 months","when she was 5 1/2 months old"]},
                            "万圣节": {"数量":1,"例子":["Easter"]}
                            }
                            ```
            '''
    messages = [{"role": "system", "content": role_function},
                {"role": "user", "content": f"以下是某款产品购买用户的{name}，大概有{len(entitty_set)}个，请按照你的角色功能设定对以下一系列描述进行处理：{enetity_set_str}"}]
    response = client_ds.create(
                                model = "deepseek-chat",
                                messages = messages,
                                temperature = 0.5,
                                response_format={'type':"json_object"})
    entity_set = json.loads(response.choices[0].message.content)
    return entity_set

# 购买时间：数量少，所以分类和数量基本准确
# 使用场景：数量少，所以分类和数量基本准确
# 0\1
idx = -1
specific_entity = entity_set_filter[idx]
print(f"非去重总量：{len(specific_entity)}")
specific_entity_nodup = list(set(specific_entity))
print(f"实际总量：{len(specific_entity_nodup)}")
entity_cluster = get_cluster("购买时间", specific_entity_nodup)
print(f"统计总量：{sum([i['数量'] for i in entity_cluster.values()])}")
entity_cluster

非去重总量：484
实际总量：339


JSONDecodeError: Expecting value: line 473 column 6 (char 17247)

In [41]:
# 用户类别：数量正确，直接对名词（形式）归类即可
# 产品关注点：数量不正确，直接对名词（形式）归类即可
# 标点符号、tokenization、stopwords、提示词 
# 2\-1
idx = 3
specific_entity = entity_set_filter[idx]
print(f"非处理数据总量：{len(specific_entity)}")
specific_entity_nopunct = list(map(remove_punct, specific_entity))
specific_entity_token = list(map(word_tokenize, specific_entity_nopunct))
specific_entity_nostopwords = list(map(remove_stopwords, specific_entity_token)) #### stopword可以优化 words = [token.text for token in doc if not token.is_stop]
specific_entity_full = []
for x in specific_entity_nostopwords:
    for y in x:
        specific_entity_full.append(y)
specific_entity_full = sorted(specific_entity_full)
specific_entity_full
specific_entity_sorted = dict(Counter(specific_entity_full))
print(f"非去重后实体总量：{len(specific_entity_full)}")
print("数量统计：", specific_entity_sorted)
print(f"去重后实体总量：{len(set(specific_entity_full))}")

非处理数据总量：139
非去重后实体总量：226
数量统计： {'Christmas': 1, 'Grandmas': 1, 'add': 1, 'alternative': 1, 'animals': 1, 'another': 1, 'baby': 12, 'back': 1, 'best': 1, 'birthday': 1, 'book': 4, 'books': 2, 'buying': 1, 'category': 1, 'charity': 1, 'children': 2, 'colour': 1, 'could': 1, 'coworker': 1, 'daughter': 1, 'develop': 1, 'development': 2, 'developmental': 1, 'dinosaurs': 1, 'early': 1, 'easily': 1, 'education': 5, 'entertainment': 1, 'first': 1, 'following': 1, 'free': 1, 'friend': 1, 'friends': 1, 'get': 1, 'gift': 90, 'gifted': 1, 'gifts': 4, 'give': 1, 'good': 2, 'got': 1, 'great': 3, 'habit': 1, 'hands': 1, 'house': 1, 'intended': 1, 'interested': 1, 'items': 1, 'learn': 1, 'learning': 2, 'listed': 2, 'little': 2, 'looking': 1, 'loved': 1, 'makes': 1, 'mom': 1, 'much': 1, 'needed': 1, 'needs': 1, 'new': 1, 'obsessed': 1, 'one': 2, 'outgrew': 1, 'part': 1, 'personal': 1, 'play': 1, 'present': 6, 'purposes': 1, 'reading': 2, 'registry': 3, 'reviews': 1, 'road': 1, 'safe': 1, 'seller': 1, '

In [43]:
def get_cluster(name, entitty_set_count):
    # entity clustering
    role_function = '''
                    角色说明：
                        你是一个擅长文字理解和数据分析的专家，现在你需要完成以下任务。

                    任务说明：
                        我会给你从某一产品的用户评论中抽取出来和购买特征相关的一系列关键词（如用户类别以及用户对产品的关注点等）以及统计数量，我需要你把形式上和语义上相似的关键词进一步归类并返回。
                    
                    操作步骤：
                    1. 阅读并理解每个关键词。
                    2. 根据关键词形式上和语义上的相似度进行归类。
                    3. 输出每个类别以及该类别的关键词和数量。

                    注意事项：
                    1. 归类时主要考虑描述的形似和语义相似性（比如说“quantity”和“number”可以归为“数量”类别，“babies”、“baby”和“infant”可以归为“婴儿”类别）。
                    2. 并且根据我给出的关键词的属性对关键词序列进行筛选，忽略无关的关键词。比如，我告诉你关键词属于“用户的类别”，关键词序列为
                    （{'baby': 150, 'old': 76, 'month': 44, 'babies': 38, 'grandson': 29, 'little': 25}），的“old”、“month”、“little”可以忽略（因为这些数量词性质上不是用户）只需要返回“baby”、“babies”、“grandson”
                    3. 保证返回的关键词和统计数量与输入一致，不能出现原序列里不存在的关键词。
                    4. 以json格式返回。  

                    示例输入：
                            ```
                            关键词属性：用户对产品的关注点
                            关键词序列：                                                
                                {'tails': 68,
                                'books': 50,
                                'price': 50,
                                'quality': 48,
                                'quantity': 48,
                                'colors': 37,
                                'size': 37,
                                'number': 34,
                                'crinkle': 28,
                                'pages': 28,
                                'color': 23,
                                'crinkly': 23,
                                'cute': 23,
                                'sound': 22,
                                'book': 19,
                                'durability': 18,
                                'colorful': 17,
                                'different': 15,
                                'material': 15,
                                ...}
                            ```

                    示例输出：
                            其中，关键词“different”与属性“用户对产品的关注点”无关，则直接忽略
                            ```
                            {
                            "书籍": {"books":50, "book":19},
                            "颜色": {"colors":37, "colorful":17},
                            "材质": {"crinkly":23, "crinkle":28, "material":15},
                            "数量": {"number":34, "quantity":48},
                            "质量": {"quality":48, "durability":18},
                            "声音": {"sound": 22, "durability":18},
                            "可爱外表": {"cute": 23, "tails":68},
                            "尺寸": {"size": 37, "durability":18},
                            "价格": {"price": 50},
                            ...
                            }
                            ```
            '''
    messages = [{"role": "system", "content": role_function},
                {"role": "user", "content": f"以下序列是某款产品的{name}（关键词属性），请按照你的角色功能设定进行处理：{entitty_set_count}（关键词序列）"}]
    response = client_ds.create(
                                model = "deepseek-chat",
                                messages = messages,
                                temperature = 0.1,
                                response_format={'type':"json_object"})
    entity_set = json.loads(response.choices[0].message.content)
    return entity_set

# 用户的类别：分类基本准确，数量基本准确 
# 用户对产品的关注点：分类基本准确，作图最好加上原评论辅助理解，数量基本正确

entity_cluster = get_cluster("用户购买动机", specific_entity_sorted)
count = 0
for x, y in entity_cluster.items():
    for name, num in y.items():
        count += num
print("实体类别："+"、".join(entity_cluster.keys()))
print(f"非去重后实体总量：{count}")
entity_cluster

实体类别：礼物、教育和发展、个人使用、其他
非去重后实体总量：139


{'礼物': {'gift': 25,
  'as a gift': 48,
  'give as a gift': 1,
  'makes a great gift': 1,
  'gifts': 1,
  'gifted to a new mom': 1,
  'for a baby shower gift': 1,
  'as little Christmas gifts': 1,
  'baby gift': 2,
  'for gifts': 1,
  'as a baby shower gift': 1,
  'as a small gift': 1,
  'gift for early development': 1,
  'to add to some baby shower gifts': 1,
  'for a present': 2,
  'as a birthday present': 1,
  'for a gift': 3,
  'as a present': 2,
  'great gift': 2,
  'as a baby gift': 1,
  'part of a gift': 1,
  'intended gift': 1,
  'gift or personal use': 1},
 '教育和发展': {'for developmental purposes': 1,
  'for sensory development': 1,
  'for sensory books that could be easily washed': 1,
  'for education': 5,
  'learning toy book': 1,
  'to learn the animals': 1,
  'to develop a good reading habit': 1},
 '个人使用': {'thought daughter outgrew it, got it for road trip': 1,
  'son is obsessed with dinosaurs': 1,
  'thought this book would be just the thing': 1,
  'safe alternative': 1,
 

In [14]:
# motive（形式太多不适合统一；和上面两个实体同样的聚类方法；3）、reason（形式太多不适合统一）
## reason：先总结要点、然后分类、然后再统计

# evaluation_reasons = ""
evaluation_reasons = []
for idx, pair in enumerate(zip(evaluation, reason)):
    # evaluation_reasons += f'{idx+1}. evluation: {pair[0]}, reason: {pair[1]}\n'
    evaluation_reasons.append(f'evluation: {pair[0]}, reason: {pair[1]}')

import json
with open("./product1_review_test_pos_neg.json", 'r', encoding='utf-8') as file:
    reason_points = json.load(file)

In [71]:
# 总结要求
def get_point(evaluation_reasons):
    role_function = r'''
                    角色说明：
                        你是一个擅长文字总结的专家，现在你需要完成以下任务。

                    任务说明：
                        我会给你一系列从某一产品的用户评论中抽取出来用户对这款产品的好评、差评、中性以及对应原因，请你简洁地用自己的语言总结出这些评论的好评点和差评点。

                    操作步骤：
                    1. 遍历阅读并理解每个评论的属性（好评、差评和中性）以及对应原因。
                    2. 总结出差评点和好评点各有哪些，数量尽量精简，并给出1~3个简短的例子。

                    注意事项：
                    1. 不能随意捏造与输入事实不符的内容。
                    2. 以json格式返回好评点/差评点。

                    示例输入：
                            ```
                            1. evluation: positive, reason: crinkle sounds and tails keep the kiddo entertained, but the reviewer is confused about the bookmarks
                            2. evluation: positive, reason: absolutely adorable and great quality
                            3. evluation: negative, reason: expected a book but received a limp cloth item that resembled a chew toy
                            4. evluation: positive, reason: child loves to play with the product
                            5. evluation: positive, reason: the user found the product nice
                            6. evluation: positive, reason: the user simply states 'GOOD'
                            7. evluation: positive, reason: daughter loves the crinkling noise and holding on to all the different tails, very durable, easy to clean
                            8. evluation: positive, reason: mommy to be is very excited for her baby to have it
                            9. evluation: negative, reason: quality of the product is not very good and there should have been something inside that made other kind of noise
                            10. evluation: negative, reason: misleading picture and overpriced
                            ...
                            ```

                    示例输出：
                            ```
                            {
                            "postive point": [
                                            "entertaining": ["crinkling", "loved by kids"],
                                            "good quality": ["cleanability", "durability"],
                                            ...
                                            ],
                            "negative point": [
                                            "misleading package": ["bookmark confusion", "misleading picture"],
                                            "unreasonable price": ["overpriced", "durability"],
                                            ...
                                            ]
                            }
                            ```
            '''
    
                            # {
                            # "postive pointe": {
                            #                 "entertainment": "crinkle sounds and tails keep the kiddo entertained",
                            #                 "great quality": "very durable",
                            #                 ...
                            #                 },
                            # "negative point": {
                            #                 "misleading appearance": "expected a book but received a limp cloth item that resembled a chew toy",
                            #                 "unreasonable price" "overpricd",
                            #                 "poor quality": "quality of the product is not very good",
                            #                 ...
                            #                 }
                            # }

    messages = [{"role": "system", "content": role_function},
                {"role": "user", "content": f"以下是一系列评论属性以及对应原因：{evaluation_reasons}，请按照你的角色功能进行处理。"}]
    response = client_ds.create(
                                model = "deepseek-chat",
                                messages = messages,
                                temperature = 0.3,
                                response_format={'type':"json_object"})
    entity_set = json.loads(response.choices[0].message.content)
    return entity_set

# grouped_points = get_point(evaluation_reasons)

# 分类
def get_point(evaluation_reason, reason_points):
    role_function = r'''
                    角色设定：
                        你是一位专业的文本分析专家，擅长从用户评论中提取和对比关键信息。

                    任务说明：
                        分析给定的用户评论，并将其与提供的好评点和差评点进行对比，以确定评论中包含的是好评点还是差评点。

                    操作步骤：
                        1. 仔细阅读并理解用户提供的真实评论和评价。
                        2. 将真实评论与好评点和差评点（以及示例）进行对比分析。
                        3. 根据分析结果，确定评论中包含的好评点或差评点。

                    注意事项：
                        1. 确保输出结果基于输入内容，不得编造与事实不符的信息。
                        2. 以JSON格式返回分析结果，指出评论中的好评点或差评点。

                    示例输入：
                            ```
                            好坏评点和示例：
                                    {'positive point': [{'entertaining': ['crinkling',
                                                            'loved by kids',
                                                            "baby loves the crinkle noise and it's easy to hold",
                                                            'baby loves the crinkle page',
                                                            'baby loves the crinkle sound']},
                                    {'good quality': ['cleanability',
                                                            'durability',
                                                            'well made',
                                                            'excellent quality',
                                                            'good quality fabric that should stand up to rough handling, chewing and multiple washes']}],
                                    'negative point': [{'misleading package': ['misleading picture',
                                                            'misleading description',
                                                            'misleading product description',
                                                            'misleading information about the number of books included for the price',
                                                            'misleading photo showing five books']},
                                    {'unreasonable price': ['overpriced',
                                                            'expensive for one book',
                                                            'not worth the price',
                                                            'high price for one book',
                                                            'expensive for just one item']}]}
                            真实评论：
                                    evluation: negative, reason: not worth the money
                            ```

                    示例输出：
                            ```
                            {"point": ['unreasonable price']}
                            ```
            '''

    messages = [{"role": "system", "content": role_function},
                {"role": "user", "content": f"以下是一条用户评价和评论：{evaluation_reason}以及好评点和差评点{reason_points}，请按照你的角色功能进行处理。"}]
    response = client_ds.create(
                                model = "deepseek-chat",
                                messages = messages,
                                temperature = 0.1,
                                response_format={'type':"json_object"})
    points = json.loads(response.choices[0].message.content)
    return points

import numpy as np
# idx = np.random.choice(list(range(len(evaluation_reasons))))
# evaluation_reason = evaluation_reasons[idx]
# points = get_point(evaluation_reason, reason_points)
# print(evaluation_reason)
# print(points)

# 统计
from tqdm import tqdm 
def get_statistics(evaluation_reasons, reason_points):
    points = []
    for idx, evaluation_reason in tqdm(enumerate(evaluation_reasons)):
        try:
            point = get_point(evaluation_reason, reason_points)
            points.append(point['point'])
        except Exception as e:
            points.append([])
            print(f"索引：{idx}；内容：{evaluation_reason}；出错原因：{e}")
    return points
points = get_statistics(evaluation_reasons, reason_points)

0it [00:00, ?it/s]

699it [15:10,  1.30s/it]


In [74]:
points_list = []
for x in points:
    for y in x:
        points_list.append(y)
from collections import Counter
Counter(points_list)

Counter({'entertaining': 229,
         'engaging features': 175,
         'misleading package': 143,
         'good quality': 139,
         'unreasonable price': 115,
         'educational value': 84,
         'interactive': 59,
         'poor quality': 52,
         'safety concerns': 11,
         'meets expectations': 1,
         'user loves the product': 1,
         'recipient was very pleased with the little book': 1,
         'product is recommended': 1,
         'ideal baby gift': 1,
         'parents seem pleased': 1})

In [ ]:
!pip freeze > requirements.txt